<a href="https://colab.research.google.com/github/RohanBolle/BigCrunch/blob/main/Big_crunch%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp
from sympy import lambdify
import cmath
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
G_0 = 1
H_0 = 1 / 13.8  # Hubble constant
c = 3e9
Omega_0_m = 5  # normalized mass density
omega_r = 10e-4
omega_lambda = 0
omega_k = 1-omega_r-omega_lambda-Omega_0_m
omega_0 = omega_lambda + omega_r + Omega_0_m

# Time settings
t_i = 0.001
t_f = 10
y_i = 0.01
r_vals = np.linspace(0.1, 10, 1000)

In [ ]:
np.sum([omega_r, omega_lambda, Omega_0_m])

In [ ]:
# Original Friedmann
sol_ori = solve_ivp(lambda t, r: r*H_0*np.sqrt(Omega_0_m/(r**3) + (omega_k)/(r**2) + omega_r/(r**4) + omega_lambda), [t_i, t_f], [y_i], t_eval=np.linspace(t_i, t_f, 1000), dense_output = False, method='RK45', rtol = 1e-3)

plt.figure(figsize=(10, 6))
plt.plot(sol_ori.t, sol_ori.y[0],  "-", color ='black', label ='Standard Friedmann')
plt.xlabel("Time (t)")
plt.ylabel("Scale factor (a)")
plt.legend()
plt.grid()
plt.show()


<ipython-input-225-3083123174>:2: RuntimeWarning: invalid value encountered in sqrt
  sol_ori = solve_ivp(lambda t, r: r*H_0*np.sqrt(Omega_0_m/(r**3) + (omega_k)/(r**2) + omega_r/(r**4) + omega_lambda), [t_i, t_f], [y_i], t_eval=np.linspace(t_i, t_f, 1000), dense_output = False, method='RK45', rtol = 1e-3)


In [ ]:
amax = sol_ori.y[0][-1]

In [ ]:
from scipy.optimize import root

def expression_under_sqrt(r):
  # The function expects a single value for r
  if isinstance(r, (list, np.ndarray)):
    r = r[0] # Take the first element if it's an array

  return Omega_0_m/(r**3) + omega_k/(r**2) + omega_r/(r**4) + omega_lambda

# We need to provide an initial guess for the root.
# Looking at the plot we made earlier, there seems to be a root around r=1.
initial_guess = 1.0

# Find the root
sol = root(expression_under_sqrt, initial_guess)

if sol.success:
  root_value = sol.x[0]
  print(f"The expression under the square root is approximately zero at r = {root_value:.4f}")
  # Based on the plot, for r greater than this value, the expression is negative.
  print(f"The square root turns negative for r values approximately greater than {root_value:.4f}")
else:
  print("Root finding failed.")


In [ ]:
# This is a conceptual example and might need adjustments
# depending on how precisely you want to handle the time axis

# Assuming sol_ori.t and sol_ori.y[0] contain the expansion data up to amax
# Find the index where the scale factor is close to amax (root_value)
root_value = 1.059
amax_index = np.argmin(np.abs(sol_ori.y[0] - root_value))

# Get the time at amax
t_at_amax = sol_ori.t[amax_index]

# Reverse the time and scale factor data from amax to the beginning
# We'll create new time values that go forward in time from t_at_amax
# The scale factor values will be the reversed values from the expansion

# Time for the Big Crunch phase (relative to t_at_amax)
big_crunch_relative_time = sol_ori.t[:amax_index][::-1] - sol_ori.t[0]

# Scale factor for the Big Crunch phase
big_crunch_scale_factor = sol_ori.y[0][:amax_index][::-1]

# Total time for the combined plot
combined_t = np.concatenate((sol_ori.t[:amax_index], t_at_amax + big_crunch_relative_time))

# Total scale factor for the combined plot
combined_y = np.concatenate((sol_ori.y[0][:amax_index], big_crunch_scale_factor))


plt.figure(figsize=(10, 6))
plt.plot(combined_t, combined_y,  "-", color ='black', label ='Standard Friedmann (Expansion and Big Crunch)')
plt.xlabel("Time (t)")
plt.ylabel("Scale factor (a)")
plt.legend()
plt.grid()
plt.show()

CHATGPT SOLUTION